In [20]:
import praw
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
import csv
import igraph as ig

reddit = praw.Reddit(
    client_id=os.getenv('reddit_client_id'),
    client_secret=os.getenv('reddit_client_secret'),
    user_agent=os.getenv('reddit_user_agent'),
    password=os.getenv('reddit_password'),
    username=os.getenv('reddit_username'))

In [21]:
submission = reddit.submission('1diog3i')
submission.comments.replace_more(limit=10)
print(len(submission.comments.list()))

152


In [22]:
class RedditResponse:
    comment_id = '-1'
    comment_body = ''
    comment_author_id = '-1'
    comment_author_name = ''
    comment_parent_id = '-1'
    comment_parten_name = ''
    comment_posted_time = ''
    comment_score = 0


    def __init__(self, comment_id, comment_body, comment_author_id, comment_author_name, comment_parent_id, comment_parent_name, comment_posted_time, comment_score):
        self.comment_id = comment_id
        self.comment_body = comment_body
        self.comment_author_id = comment_author_id
        self.comment_author_name = comment_author_name
        self.comment_parent_id = comment_parent_id
        self.comment_parent_name = comment_parent_name
        self.comment_posted_time = comment_posted_time
        self.comment_score = comment_score

    def to_dict(self):
        return {
            'comment_id': self.comment_id,
            'comment_author_id': self.comment_author_id,
            'comment_author_name': self.comment_author_name,
            'comment_parent_id': self.comment_parent_id,
            'comment_parten_name': self.comment_parent_name,
            'comment_body': self.comment_body,
            'comment_posted_time': self.comment_posted_time,
            'comment_score': self.comment_score
        }

In [23]:
reddit_comments_dict_list = []
i = 0
c = len(submission.comments.list())
for comment in submission.comments.list():
    if comment.parent() is not None:
        try:
            r = RedditResponse(
                comment_id = comment.id,
                comment_body = comment.body,
                comment_author_id = comment.author.id,
                comment_author_name = comment.author.name,
                comment_parent_id = comment.parent().author.id,
                comment_parent_name = comment.parent().author.name,
                comment_posted_time = comment.created_utc,
                comment_score = comment.score
            )
            reddit_comments_dict_list.append(r.to_dict())
            print(f'{i}/{c}', end='\r')
            i += 1
        except:
            print('error')
            continue

df = pd.DataFrame(reddit_comments_dict_list)
df.to_csv('reddit_comments.csv', index=False)